<a href="https://colab.research.google.com/github/park-hoyeon/park-hoyeon.github.io/blob/master/skt_7_10_%EB%94%A5%EB%9F%AC%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models # 파이토치에서 제공하는 이미지 분야에 특화된 모델들을 불러옴
# 미리 학습되지 않은(랜덤 초기화) ResNet-50
resnet50 = models.resnet50(pretrained=False)
# pretrained=False로 설정하면, 모델의 가중치(파라미터)가 무작위로 초기화되어 있어,
# 즉, 아직 학습이 전혀 되지 않은 상태이므로, 처음부터 다시 학습(train)해야 함
# 미리 학습된(pretrained) ResNet-50
resnet50_pretrained = models.resnet50(pretrained=True)
# pretrained=True로 설정하면, ImageNet 데이터셋(대략 100만 장 이상의 이미지, 1000개 클래스)으로 미리 학습된 가중치를 그대로 불러옴
# 미리 학습된 모델을 사용하면, 적은 데이터로도 좋은 성능을 얻을 가능성이 높아짐. 이미 ‘일반적인 이미지 특징’을 잘 학습하고 있기 때문
# 마지막 FC 레이어 수정 (예: 클래스 수 10개로 변경)
num_features = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_features, 10)

# Bottleneck 블록 직접 구현하기


In [ ]:
import torch.nn as nn # 신경망 네트워크(레이어, 손실 함수 등)를 구성하는 기본 모듈
import torch.nn.functional as F # PyTorch에서 자주 쓰는 함수들(functional API). 예: F.relu, F.conv2d 등.
class Bottleneck(nn.Module):
    expansion = 4  # Bottleneck 확장 배수 (마지막 1x1 Conv를 통해 채널 수를 4배로 확장한다는 설정값)
    # in_channels: 블록의 입력 특징 맵(feature map)의 채널 수. (이전 블록의 출력 채널 수)
    # out_channels: 1x1과 3x3를 처리한 후, 마지막 단계에서 확장되기 전 기본 채널 수.
    # stride: 합성곱의 보폭. 보통은 1이지만, 블록에 따라 2로 설정되어 크기를 절반으로 줄이는 역할을 할 수도 있음.
    # downsample: skip connection에서 입력(identity)의 차원을 맞춰주기 위한 추가 모듈(합성곱+BN 등)이 들어갈 수 있음.
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        # 부모 클래스(nn.Module)의 초기화 메서드를 호출해 필요한 내부 구조를 설정
        # 1x1 Conv
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        # bias=False: 합성곱 후에 BatchNorm을 바로 쓰면, 합성곱의 bias가 불필요하다고 판단하여 생략하는 경우가 많음
        self.bn1 = nn.BatchNorm2d(out_channels) # Batch Normalization으로 학습을 안정화하고, 학습 속도를 높여준다. 왜?
        # 3x3 Conv
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        # 1x1 Conv (채널 확장, 출력 채널을 out_channels * expansion(기본 4배)로 확장)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion,
                               kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        # ResNet의 Bottleneck 구조는 이렇게 첫 번째 1x1 Conv에서 채널을 줄였다가(연산량 감소 효과),
        # 3x3에서 특징 추출 후, 마지막에 다시 1x1 Conv로 채널을 확장
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        # skip connection에서 입력 x와 출력 out의 차원(채널 수 및 가로/세로 크기)이 다를 경우,
        # 이 값이 합성곱+BN 모듈 등이 되어 identity를 변환해 줌.
        # (예를 들어 블록의 stride=2로 크기가 절반이 되면, identity도 크기를 맞춰야 하기 때문)
    def forward(self, x): # 블록에서 입력 x가 들어왔을 때, 어떤 계산을 할지 정의
        identity = x # skip connection에 사용할 원본 입력을 identity라는 변수에 잠시 저장
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)
        # downsample이 있으면 skip connection의 차원을 맞춤
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity # skip connection(잔차 연결)
        # ResNet의 핵심 아이디어는 입력(x)을 출력에 더해 모델이 잔차(residual) 만을 학습하도록 한 것
        out = self.relu(out)
        return out

이 Bottleneck 코드는 이미지가 들어오면 채널을 줄였다가(conv1), 특징을 뽑고(conv2), 다시 채널을 늘린 다음(conv3), 마지막에 원래 입력 이미지를 더해주는(residual connection) 과정을 거치는 작은 신경망 블록을 만든 *것*

###  컨볼루션 레이어 다음에 Batch Normalization(배치 정규화)을 사용하는 것
: 신경망은 여러 층(레이어)을 쌓아서 만듭니다. 각 층은 이전 층의 출력을 입력으로 받아서 계산을 합니다.
학습 과정에서 각 층의 파라미터(가중치, 편향)는 계속 변합니다.
문제는 이전 층의 파라미터가 변하면, 현재 층으로 들어오는 입력 데이터의 분포(평균, 분산 등)가 계속 바뀌게 됩니다. 이게 마치 데이터의 '공변량'이 변하는 것 같다고 해서 '내부 공변량 변화'라고 부릅니다.
이렇게 층마다 입력 데이터의 분포가 계속 바뀌면, 현재 층은 매번 새로운 분포에 맞춰서 학습을 해야 하므로 학습 속도가 느려지고, 학습이 불안정해지거나 심지어 수렴하지 못하는 경우도 생깁니다.

--> 결론적으로, 컨볼루션 레이어 다음에 Batch Normalization을 사용하는 것은 컨볼루션 연산의 결과를 안정화시켜서, 다음 레이어가 더 일관된 데이터를 받아 효율적이고 안정적으로 학습할 수 있도록 돕기 위함

# ResNet-50 전체 구조 구현

In [ ]:
class MyResNet50(nn.Module):
    def __init__(self, num_classes=1000):
        super(MyResNet50, self).__init__()
        # 초기 Stem (보통 ResNet에서는 첫 번째 7x7 Conv + MaxPool을 거쳐 이미지 크기를 빠르게 줄이는 과정을 Stem이라고 부름)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # inplanes(현재 블록의 입력 채널 수를 추적하는 변수) 설정
        # 처음에는 64(Stem의 출력 채널)로 설정.
        self.inplanes = 64
        # 레이어 구성
        self.layer1 = self._make_layer(Bottleneck, 64,  3, stride=1)
        self.layer2 = self._make_layer(Bottleneck, 128, 4, stride=2)
        self.layer3 = self._make_layer(Bottleneck, 256, 6, stride=2)
        self.layer4 = self._make_layer(Bottleneck, 512, 3, stride=2)
        # 분류기(Head) 부분
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1)) # 입력 Feature Map의 크기와 상관없이 (1, 1) 크기로 만듦
        self.fc = nn.Linear(512 * Bottleneck.expansion, num_classes)
    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        # stride!=1이거나, 채널 수가 맞지 않으면 다운샘플 구성
        # 다운샘플(downsample): stride가 2가 되면(또는 채널 수가 달라지면),
        # skip connection에서 입력 x와 출력의 크기를 맞춰줘야 함
        # downsample은 1x1 Conv와 BN으로 구성되며, stride를 조절해 공간 크기를 줄이거나, 채널 수를 맞춰줌
        if stride != 1 or self.inplanes != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, out_channels * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion)
            )
        layers = []
        layers.append(block(self.inplanes, out_channels, stride, downsample)) # 입력이 줄어들거나 채널이 변경되는 지점
        self.inplanes = out_channels * block.expansion # self.inplanes를 업데이트 (새로운 블록의 출력 채널)
        # 나머지 블록은 stride=1 이므로 downsample 없이 이어짐
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, out_channels))
        return nn.Sequential(*layers) # 마지막에 nn.Sequential(*layers)로 묶어 하나의 큰 레이어로 반환
    def forward(self, x):
        # Stem
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
          # 4개의 레이어
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        # 분류
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x
# 모델 예시 생성 (최종 출력)
model = MyResNet50(num_classes=1000)
print(model)


# UNet Model


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from skimage.draw import disk, rectangle

In [ ]:
# Function to generate an image with shapes and labels
def create_image_with_shapes_and_labels(image_size=(256, 256)):
    """
    Creates a dummy RGB image with shapes (circle, rectangle) and corresponding labels.
    Args:
        image_size (tuple): Size of the image (H, W).
    Returns:
        torch.Tensor: RGB image tensor (3, H, W).
        torch.Tensor: Label tensor (H, W) with classes 0 (background), 1 (circle), 2 (rectangle).
    """
    image = np.zeros((*image_size, 3), dtype=np.float32)  # 높이 × 너비 × RGB(3채널)로 된 검정색 빈 이미지 생성
    label = np.zeros(image_size, dtype=np.int64)  # 같은 크기의 라벨(정답) 배열도 만듦. 초기값은 전부 배경 (0)
    # Draw a circle
    rr, cc = disk((64, 64), 40)
    image[rr, cc, 0] = 1.0  # Red circle
    label[rr, cc] = 1  # Class 1: Circle
    # Draw a rectangle
    start = (120, 120)
    extent = (50, 80)
    rr, cc = rectangle(start=start, extent=extent)
    image[rr, cc, 1] = 1.0  # Green rectangle
    label[rr, cc] = 2  # Class 2: Rectangle
    # Normalize to range [0, 1] - 이미지 정규화
    image = (image - image.min()) / (image.max() - image.min())
    return torch.tensor(image).permute(2, 0, 1), torch.tensor(label)  # (C, H, W), (H, W)

In [ ]:
# Define a basic double convolution block
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False), # 패딩이 1이라서 이미지 크기는 유지됨
        nn.BatchNorm2d(out_channels),# 출력 채널별로 정규화해줌 → 학습 안정화, 속도 향상


        nn.ReLU(inplace=True),# 비선형성 부여
        nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True), # 출력 채널 수를 유지하면서 한 번 더 특징을 뽑아냄 (이중 컨볼루션으로 더 깊은 특성 추출 가능)
    )


결과적으로 이 블록은?<br>
3×3 conv 2번 → receptive field가 커짐 (간접적으로 5×5 효과)<br>

BatchNorm + ReLU 2번 → 안정적이고 표현력 있는 특징 생성<br>

U-Net에서 반복 사용되며, Encoder와 Decoder 모두에 핵심 구성 요소<br>

In [ ]:
# Define the U-Net model
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        # Encoder - 채널 수를 점점 늘려가며 깊은 특징 추출
        self.enc1 = double_conv(in_channels, 64)
        self.enc2 = double_conv(64, 128)
        self.enc3 = double_conv(128, 256)
        self.enc4 = double_conv(256, 512)

        # Bottleneck - 가장 깊은 특징 추출 구간 (이미지 크기 가장 작음)
        # U-Net 중앙 - 채널 수가 가장 큼
        self.bottleneck = double_conv(512, 1024)

        # Decoder - Encoder의 출력을 cat으로 이어붙여 경계 보존 + 이후 다시 double_conv로 정제
        # ConvTranspose2d: 업샘플링 (해상도 2배 증가)
        # double_conv: 합친 결과를 다시 정제 (잡음 제거, 경계 보존)
        # torch.cat(enc, dec): 같은 레벨의 인코더 출력과 합침
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec4 = double_conv(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = double_conv(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = double_conv(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = double_conv(128, 64)

        # Output layer - 마지막 채널 수를 클래스 수로 줄임 (예: 3개의 클래스라면 3 채널)
        self.out_conv = nn.Conv2d(64, out_channels, kernel_size=1)
    def forward(self, x):

        # Encoder - Pooling으로 해상도는 줄이고, 채널은 증가
        enc1 = self.enc1(x)
        enc2 = self.enc2(nn.MaxPool2d(kernel_size=2)(enc1))
        enc3 = self.enc3(nn.MaxPool2d(kernel_size=2)(enc2))
        enc4 = self.enc4(nn.MaxPool2d(kernel_size=2)(enc3))
           # Bottleneck
        bottleneck = self.bottleneck(nn.MaxPool2d(kernel_size=2)(enc4))

        # Decoder - 업샘플링 (ConvTranspose2d) → 인코더의 같은 단계와 concat -> 경계 정보 복원, 더 세밀한 결과 생성
        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((enc4, dec4), dim=1) # upconv4: Transposed Convolution을 사용해 업샘플링 (크기 2배로 키움)
        dec4 = self.dec4(dec4) # torch.cat: 인코더(enc4)의 출력과 디코더(dec4)를 채널 방향으로 이어 붙임
        dec3 = self.upconv3(dec4) # dec4: double_conv 블록으로 특징 다시 정리 (경계 보존 + 채널 축소)
        dec3 = torch.cat((enc3, dec3), dim=1)
        dec3 = self.dec3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((enc2, dec2), dim=1)
        dec2 = self.dec2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((enc1, dec1), dim=1)
        dec1 = self.dec1(dec1)

        # Output - 픽셀마다 클래스 score 예측 → 최종 마스크 결과
        out = self.out_conv(dec1)
        return out


① Encoder (인코더)	이미지를 점점 작게 만들면서 특징 추출 <br>
② Bottleneck (중앙 통로)	정보 압축 구간 (특징 요약) <br>
③ Decoder (디코더)	이미지를 다시 원래 크기로 복원 (픽셀별 예측) <br>

### 🔽 디코더 레이어 4 (가장 깊은 층부터 복원 시작)
dec4 = self.upconv4(bottleneck)  # 1024 → 512 채널로 업샘플링 (공간 크기 2배) <br>
dec4 = torch.cat((enc4, dec4), dim=1)  # enc4와 연결: 채널 512 + 512 → 1024 <br>
dec4 = self.dec4(dec4)  # 1024 → 512 채널로 다시 정리<br>


In [ ]:
# Training loop
def train_model(model, optimizer, criterion, num_epochs, input_image, ground_truth):
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad() # 이전 step에서 계산된 gradient 초기화
        # Forward pass
        outputs = model(input_image) #이미지 입력 → 모델이 예측한 마스크 outputs 생성
        loss = criterion(outputs, ground_truth.unsqueeze(0))  # ground_truth에 .unsqueeze(0)을 적용해 배치 차원 (1, H, W)로 만듦
        # Backward pass - 손실을 모델 파라미터로 미분
        loss.backward()
        optimizer.step() # 계산된 gradient를 바탕으로 모델 가중치를 한 번 업데이트
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")
    print("Training complete!")


In [ ]:
# Visualization of results
def visualize_results(input_image, output_prediction, ground_truth=None):
    # 배치 차원 제거 (squeeze()) → (3, H, W)
    # 채널 순서 변경 (permute) → (H, W, 3) → 이미지로 보이게 함
    # cpu().numpy()로 넘파이 배열로 변환
    input_image = input_image.squeeze().permute(1, 2, 0).cpu().numpy()  # Convert to HWC

    # argmax로 가장 높은 확률의 클래스 선택 -> 각 픽셀에 대해 가장 가능성 높은 클래스 번호만 추출
    output_prediction = torch.argmax(output_prediction, dim=1).squeeze().cpu().numpy()  # Convert to label map
    if ground_truth is not None:
        ground_truth = ground_truth.cpu().numpy()

    # Plot the images - 1행 3열의 subplot 만들기 (이미지, 예측, 정답)
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    # 첫 번째: 원본 이미지 보여주기
    ax[0].imshow(input_image)
    ax[0].set_title("Input Image")
    ax[0].axis("off")

    # 두 번째: 모델이 예측한 분할 결과 (클래스별 색) 보여주기
    ax[1].imshow(output_prediction, cmap="jet")
    ax[1].set_title("Model Prediction")
    ax[1].axis("off")
    if ground_truth is not None: # 세 번째: 정답 레이블이 있는 경우 같이 비교해보기
        ax[2].imshow(ground_truth, cmap="jet")
        ax[2].set_title("Ground Truth")
        ax[2].axis("off")
    plt.tight_layout()
    plt.show()

In [ ]:
# Example usage
if __name__ == "__main__":
    # Create synthetic data
    input_image, ground_truth = create_image_with_shapes_and_labels()
    input_image = input_image.unsqueeze(0)  # Add batch dimension - PyTorch는 (Batch, Channel, Height, Width) 순서를 기대하므로 unsqueeze(0)으로 배치 차원 추가 → (1, 3, 256, 256)
    ground_truth = ground_truth  # (H, W)
    # Instantiate U-Net model
    num_classes = 3  # Background, Circle, Rectangle
    #model = UNet(in_channels=3, out_channels=num_classes)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 입력 채널은 RGB(3), 출력 채널은 클래스 수(배경, 원, 사각형 = 3)
    model = UNet(in_channels=3, out_channels=num_classes).to(device)
    input_image = input_image.to(device)
    ground_truth = ground_truth.to(device)

    # Define loss and opt
    criterion = nn.CrossEntropyLoss() # CrossEntropyLoss: 다중 클래스 분류 문제에서 사용
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the model
    train_model(model, optimizer, criterion, num_epochs=10, input_image=input_image, ground_truth=ground_truth)

    # Evaluate the model - .eval()은 드롭아웃/배치정규화 등을 평가 모드로 전환
    model.eval()
    with torch.no_grad():
        output_prediction = model(input_image)
    # Visualize results
    visualize_results(input_image.squeeze(), output_prediction, ground_truth) #squeeze()는 (1, 3, H, W) → (3, H, W)로 배치 차원 제거